# Monte Carlo Methods
In this notebook, you will write your own implementations of many Monte Carlo(MC) algorithms.
While we have provided some starter code, you are welcome to erase these hints and write code from scratch

## Part 0: Explore BlackjackEnv
We begin by importing the necessary packages.

In [1]:
import sys
import gym
import numpy as np
from collections import defaultdict

from plot_utils import plot_blackjack_values, plot_policy

Use the code cell below to create an instance of the Blackjack environment.

In [2]:
env = gym.make('Blackjack-v0')

In [3]:
print(env.observation_space)

Tuple(Discrete(32), Discrete(11), Discrete(2))


In [4]:
print(env.action_space)

Discrete(2)


In [8]:
for i_episode in range(3):
  state = env.reset()
  while True:
    print(state)
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    if done:
      print('End game! Reward: ', reward)
      print('You won :)\n') if reward > 0 else print('You lost :(\n')
      break

(6, 10, False)
(16, 10, False)
End game! Reward:  -1
You lost :(

(14, 10, True)
(15, 10, True)
(17, 10, True)
(21, 10, True)
(13, 10, False)
End game! Reward:  -1.0
You lost :(

(14, 8, False)
(17, 8, False)
End game! Reward:  -1
You lost :(



## Part 1: MC Prediction
In this section, you will write your own implementation of MC predction (for estimating the action-value function).

We will begin by investigating a policy where the player almost sticks if the sum of her cards exceeds 18. In particular, she select action STICK with 80% probability if the sum is greater than 18; and, if the sum is 18 or below, she selects action HIT with 80% probability. The function generate_episode_from_limit_stochastic samples an episode using this policy.

It returns as output:
* episode: This is a list of (state, action, reward) tuples (of tuples) and corresponds to $(S_0, A_0, R_1, \ldots, S_{T-1}, A_{T-1}, R_{T})$, where $T$ is the final time step. In particular, `episode[i]` returns $(S_i, A_i, R_{i+1})$, and `episode[i][0]`, `episode[i][1]`, and `episode[i][2]` return $S_i$, $A_i$, and $R_{i+1}$, repectively.

In [13]:
def generate_episode_from_limit_stochastic(bj_env):
  episode = []
  state = bj_env.reset()
  while True:
    probs = [0.8, 0.2] if state[0] > 18 else [0.2, 0.8]
    action = np.random.choice(np.arange(2), p=probs)
    next_state, reward, done, info = bj_env.step(action)
    episode.append((state, action, reward))
    state = next_state
    if done:
      break;
  return episode

In [14]:
for i in range(3):
  print(generate_episode_from_limit_stochastic(env))

[((14, 10, False), 1, 0), ((21, 10, False), 0, 1.0)]
[((11, 8, False), 1, 0), ((21, 8, False), 0, 1.0)]
[((17, 8, False), 1, -1)]


In [22]:
arr = np.array([1,2,3,4,5])
arr2 = np.array([1,2,3,4,5,6])
print(arr[3:], arr2[:-4], arr[2:], arr2[:-3])


[4 5] [1 2] [3 4 5] [1 2 3]


In [38]:
def mc_prediction_q(env, num_episodes, generate_episode, gamma=1.0):
  # initialize empty dictionaries of arrays
  returns_sum = defaultdict(lambda: np.zeros(env.action_space.n))
  N = defaultdict(lambda: np.zeros(env.action_space.n))
  Q = defaultdict(lambda: np.zeros(env.action_space.n))
  # loop over episodes
  for i_episode in range(1, num_episodes+1):
    # monitor progress
    if i_episode % 1000 == 0:
      print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
      sys.stdout.flush()
    # generate an episode
    episode = generate_episode(env)
    if i_episode % 1000 == 0:
      print(episode[:2])
    # obtain the states, actions, and rewards
    states, actions, rewards = zip(*episode)
    # prepare for discounting
    discounts = np.array([gamma**i for i in range(len(rewards)+1)])
    # update the sum of the returns, number of visits, and action-value
    # function estimates for each state-action pair in the episode
    for i, state in enumerate(states):
      returns_sum[state][actions[i]] += sum(rewards[i:]*discounts[:-(1+i)])
      N[state][actions[i]] += 1.0
      Q[state][actions[i]] = returns_sum[state][actions[i]] / N[state][actions[i]]
  return Q

In [39]:
# obtain the action-value function
Q = mc_prediction_q(env, 50000, generate_episode_from_limit_stochastic)

# obtain the corresponding state-value functrion


Episode 1000/50000.[((6, 8, False), 1, 0), ((10, 8, False), 1, 0)]
Episode 2000/50000.[((14, 10, False), 1, 0), ((15, 10, False), 1, -1)]
Episode 3000/50000.[((20, 10, False), 1, -1)]
Episode 4000/50000.[((14, 7, False), 1, -1)]
Episode 5000/50000.[((15, 10, False), 1, -1)]
Episode 6000/50000.[((13, 10, False), 1, 0), ((21, 10, False), 0, 1.0)]
Episode 7000/50000.[((17, 3, True), 1, 0), ((21, 3, True), 1, 0)]
Episode 8000/50000.[((20, 9, False), 0, 1.0)]
Episode 9000/50000.[((17, 6, False), 1, -1)]
Episode 10000/50000.[((16, 10, False), 1, -1)]
Episode 11000/50000.[((15, 6, False), 1, -1)]
Episode 12000/50000.[((11, 10, False), 1, 0), ((21, 10, False), 0, 1.0)]
Episode 13000/50000.[((14, 8, False), 0, -1.0)]
Episode 14000/50000.[((10, 4, False), 1, 0), ((12, 4, False), 1, 0)]
Episode 15000/50000.[((12, 7, False), 0, -1.0)]
Episode 16000/50000.[((15, 2, False), 1, 0), ((20, 2, False), 0, -1.0)]
Episode 17000/50000.[((12, 9, False), 1, -1)]
Episode 18000/50000.[((19, 9, True), 1, 0), ((1